<b><h1>AI4Code - Exploratory Data Analysis</h1></b>
![AI4Code](https://storage.googleapis.com/kaggle-competitions/kaggle/35887/logos/header.png?t=2022-05-09-22-33-02)<br>

The goal of this competition is to understand the relationship between code and comments in Python notebooks. You are challenged to reconstruct the order of markdown cells in a given notebook based on the order of the code cells, demonstrating comprehension of which natural language references which code.

### Importing data and libraries

In [ ]:
# importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns

#importing dataset
Base_dir = "../input/AI4Code/"
train_orders = pd.read_csv(os.path.join(Base_dir, "train_orders.csv"))
train_ancestors = pd.read_csv(os.path.join(Base_dir, "train_ancestors.csv")) 
train = pd.read_csv("../input/ai4code-train-dataframe/train.csv")
train.drop(["Unnamed: 0"], axis=1, inplace=True)
test = pd.read_csv(os.path.join(Base_dir, "sample_submission.csv"))

In [ ]:
# preparing data_orders.csv
dict_id_type = dict(zip(train.cell_id, train.cell_type))
dict_id_source = dict(zip(train.cell_id, train.source))
train_orders.cell_order = train_orders.cell_order.apply(lambda x: x.split())
train_orders = train_orders.explode("cell_order")
train_orders["cell_type"] = train_orders.cell_order.map(dict_id_type)
train_orders["source"] = train_orders.cell_order.map(dict_id_source)
del dict_id_type, dict_id_source

In [ ]:
# to help with the code vs cell statistics
cell_type_with_orders = train_orders.groupby("id").agg({"cell_type": lambda x: x.tolist()})
cell_type_with_orders["first_type"] = cell_type_with_orders["cell_type"].apply(lambda x: x[0])
cell_type_with_orders["second_type"] = cell_type_with_orders["cell_type"].apply(lambda x: x[0])
cell_type_with_orders["number_markdown"] = cell_type_with_orders["cell_type"].apply(lambda x: x.count('markdown'))
cell_type_with_orders["number_code"] = cell_type_with_orders["cell_type"].apply(lambda x: x.count('code'))

In [ ]:
# to help with the source code statistics
cell_source_with_orders = train_orders.groupby("id").agg({"source": lambda x: x.tolist()})
cell_source_with_orders["first_codeblock"] = cell_source_with_orders.source.apply(lambda x: x[0])

### Number of notebooks in train and test dataset

In [ ]:
print("No. of jupyter notebooks in train dataset: ", len(train_orders))
print("No. of jupyter notebooks in test dataset: ", len(test))

### Statistics around cell_types

In [ ]:
sns.countplot(train["cell_type"])

In [ ]:
print("Median code cells in train dataset: ", cell_type_with_orders["number_code"].median())
print("Median markdown cells in train dataset: ", cell_type_with_orders["number_markdown"].median())

Insight :- There are a lot more code cells than markdown cells

In [ ]:
print("Most common first cell type: ", cell_type_with_orders["first_type"].mode()[0])
print("Number of markdown first cell: ", len(cell_type_with_orders[cell_type_with_orders["first_type"] == "markdown"]))
print("Number of code first cell: ", len(cell_type_with_orders[cell_type_with_orders["first_type"] == "code"]))

Insight :- Even though code cells are more common than markdown cells. Markdown cells are more commonly the first cell.

In [ ]:
for i in range(5):
    print(cell_source_with_orders["first_codeblock"].sample(1)[0])
    print("*****************************************************")

Insight - In a lot of these cases, after looking through a lot of them, there are two possibilities<br>
- Either it is a code cell, in that case there are imports, usually the one that kaggle gives by default, import pandas as pd, import numpy as np, import os
- Or it is the title of the notebook, with either some description or just the title

In [ ]:
cell_type_with_orders["first_type_markdown"] = cell_type_with_orders.first_type.apply(lambda x: x=="markdown")

In [ ]:
cell_type_with_orders[['first_type_markdown', 'number_markdown']].corr()

## Work in progress
If you find this useful, do upvote